In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Fetching Data

In [2]:
user = 'akshayparate'
passw = 'akshayparate'
host =  '182.48.201.235'
port = 3306
database = 'stock_fundamentals'
# Create an engine instance
engine = create_engine(f'mysql+pymysql://{user}:{passw}@{host}:{port}/{database}')

In [3]:
balancesheet_df = pd.read_sql('SELECT * FROM {}'.format("balancesheet"), con=engine)

In [4]:
cashflow_df = pd.read_sql('SELECT * FROM {}'.format("cashflow"), con=engine)


In [5]:
incomesheet_df = pd.read_sql('SELECT * FROM {}'.format("incomesheet"), con=engine)

In [6]:
balancesheet_df.index = balancesheet_df["Year_Date"]
incomesheet_df.index = incomesheet_df["Year_Date"]
cashflow_df.index = cashflow_df["Year_Date"]

In [7]:
balancesheet_df.columns

Index(['OtherEquityTotal', 'PreferredStock_NonRedeemableNet', 'NetLoans',
       'AccumulatedDepreciationTotal', 'Cash', 'TotalCommonSharesOutstanding',
       'NotesPayable_ShortTermDebt', 'Property_Plant_EquipmentTotal_Gross',
       'AccountsReceivable_TradeNet', 'OtherCurrentAssetsTotal',
       'OtherLiabilitiesTotal', 'TotalReceivablesNet',
       'TotalCurrentLiabilities', 'AdditionalPaid_InCapital',
       'RedeemablePreferredStockTotal', 'AccountsPayable',
       'CashandShortTermInvestments', 'TotalDeposits',
       'Property_Plant_EquipmentTotal_Net', 'LongTermInvestments',
       'IntangiblesNet', 'ESOPDebtGuarantee', 'NoteReceivable_LongTerm',
       'PrepaidExpenses', 'Payable_Accrued', 'Annual_Quarter', 'GoodwillNet',
       'TotalLiabilitiesShareholders'Equity', 'TotalLiabilities',
       'PolicyLiabilities', 'CommonStockTotal', 'TotalUtilityPlantNet',
       'CurrentPortofLTDebt_CapitalLeases', 'ShortTermInvestments',
       'OtherCurrentliabilitiesTotal', 'TotalLongTe

In [8]:
i_df = incomesheet_df[incomesheet_df["stockName"] == "State Bank Of India"]
b_df = balancesheet_df[balancesheet_df["stockName"] == "State Bank Of India"]
c_df = cashflow_df[cashflow_df["stockName"] == "State Bank Of India"]

In [9]:
piotroski_dict = {}

### ROA

In [10]:
roa_df = pd.merge(i_df["NetIncomeBeforeExtraItems"], b_df["TotalAssets"], left_index=True, right_index=True)
roa_df.dropna(inplace=True)
roa_df = roa_df.astype(float)
roa_df["roa"] = roa_df["NetIncomeBeforeExtraItems"]/roa_df["TotalAssets"]

In [11]:
roa_df

,NetIncomeBeforeExtraItems,TotalAssets,roa
Year_Date,,,
2024,67084.6700,6.733779e+06,0.009962
2023,55648.1728,5.954418e+06,0.009346
2022,35373.8798,5.360884e+06,0.006599
2021,22405.4576,4.845619e+06,0.004624
2020,22934.1722,4.197492e+06,0.005464
2019,2299.6391,3.888464e+06,0.000591
2024-03-31,21384.1500,6.733779e+06,0.003176
2023-09-30,16099.5800,6.317053e+06,0.002549
2023-03-31,18093.8428,5.954418e+06,0.003039


### CFO

In [22]:
cfo_df = pd.merge(b_df["TotalAssets"],c_df['CashfromOperatingActivities'], left_index=True, right_index=True)
cfo_df = cfo_df.astype(float)
cfo_df["cfo"] = cfo_df["CashfromOperatingActivities"] / cfo_df["TotalAssets"]

In [23]:
cfo_df

,TotalAssets,CashfromOperatingActivities,cfo
Year_Date,,,
2024,6.733779e+06,21632.4300,0.003213
2023,5.954418e+06,-86013.6796,-0.014445
2022,5.360884e+06,57694.8464,0.010762
2021,4.845619e+06,89918.9301,0.018557
2020,4.197492e+06,23928.5317,0.005701
2019,3.888464e+06,29556.0050,0.007601
2024-03-31,6.733779e+06,21632.4300,0.003213
2023-12-31,NaN,NaN,NaN
2023-09-30,6.317053e+06,54713.7600,0.008661


### Delta ROA

In [49]:
yearList = []
delta_roa_df = pd.DataFrame()
for date in roa_df.index:
    if len(date) == 4:
        yearList.append(date)
    else:
        pass
delta_roa = []
for idx in range(0,len(yearList)-1):
    delta_roa.append(roa_df.loc[yearList[idx]]["roa"] - roa_df.loc[yearList[idx+1]]["roa"])
yearList = list(map(int, yearList))
delta_roa_df["year"] = yearList[:-1]
delta_roa_df["delta_roa"] = delta_roa

In [50]:
delta_roa_df

,year,delta_roa
0,2024,0.000617
1,2023,0.002747
2,2022,0.001975
3,2021,-0.000840
4,2020,0.004872


### Accrual

In [55]:
accrual_df = pd.DataFrame()
accrual_df["roa"] = roa_df["roa"]
accrual_df["cfo"] = cfo_df.dropna()["cfo"]
accrual_df["accrual"] = roa_df["roa"] - cfo_df.dropna()["cfo"]

In [56]:
accrual_df

,roa,cfo,accrual
Year_Date,,,
2024,0.009962,0.003213,0.006750
2023,0.009346,-0.014445,0.023791
2022,0.006599,0.010762,-0.004164
2021,0.004624,0.018557,-0.013933
2020,0.005464,0.005701,-0.000237
2019,0.000591,0.007601,-0.007010
2024-03-31,0.003176,0.003213,-0.000037
2023-09-30,0.002549,0.008661,-0.006113
2023-03-31,0.003039,-0.014445,0.017484
